### Udacity Self-Driving Car Nanodegree
# Image augmenation for Capstione Project

Find examples at the end of notebook

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
source = '/Users/jakobkammerer/Google Drive/Happy Robots/TrafficLightData_real/data000155.png'

## Image loading

In [ ]:
img = cv2.imread(source)

plt.imshow(bgr2rgb(img))
plt.show()

## Augmentation Functions

In [ ]:
def bgr2rgb(img_bgr):
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    return img_rgb

def aug_brightness(img_in):
    """
    Augment picture in brightness
    :param img_in: image in BGR (openCV standard)
    :return: image in BGR, augmented in brightness
    """
    # Change to HSV color space and transform to float64 array
    img_out = cv2.cvtColor(img_in, cv2.COLOR_BGR2HSV)
    img_out = np.array(img_out, dtype=np.float64)
    # Set a random number for brightness adjustment and adjust
    rand_bright = 0.5 + np.random.uniform()
    img_out[:,:,2] = img_out[:,:,2] * rand_bright
    # set every value higher than 255 to 255
    img_out[:,:,2][img_out[:,:,2] > 255] = 255
    img_out[:,:,2][img_out[:,:,2] < 0] = 0
    # Transform back (unit8 and BRG)
    img_out = np.array(img_out, dtype=np.uint8)
    img_out = cv2.cvtColor(img_out, cv2.COLOR_HSV2BGR)
    return img_out

def aug_shadow(img_in):
    # Random values according to image size
    width = np.shape(img_in)[1]
    height = np.shape(img_in)[0]
    top_y = width * np.random.uniform()
    bot_y = width * np.random.uniform()
    top_x = 0
    bot_x = height
    
    # Color space and mask
    img_hls = cv2.cvtColor(img_in, cv2.COLOR_BGR2HLS)
    
    shadow_mask = 0 * img_hls[:,:,1]
    X_m = np.mgrid[:height, :width][0]
    Y_m = np.mgrid[:height, :width][1]
    shadow_mask[((X_m - top_x)*(bot_y - top_y) - (bot_x - top_x) * (Y_m - top_y)) >= 0] = 1
    
    # Augment
    # Random bright .25 and .75
    #rand_bright = np.random.randint(low=8, high=7) / 100
    #rand_bright = 0.5 + np.random.uniform()
    rand_bright = (8 + (1.5 * np.random.uniform())) / 10

    cond1 = shadow_mask == 1
    cond0 = shadow_mask == 0
    if np.random.randint(2) == 1:
        img_hls[:,:,1][cond1] = img_hls[:,:,1][cond1] * rand_bright
    else:
        img_hls[:,:,1][cond0] = img_hls[:,:,1][cond0] * rand_bright
    
    img_hls[:,:,1][img_hls[:,:,1] > 255] = 255
    img_hls[:,:,1][img_hls[:,:,1] < 0] = 0
    
    img_hls = np.array(img_hls, dtype=np.uint8)
    
    # Transform back to BGR Color Space
    img_out = cv2.cvtColor(img_hls, cv2.COLOR_HLS2BGR)
    
    return img_out
        
def aug_blur(img_in):
    """
    Blurs the image
    :param img_in: BGR image (cv2 standard)
    :return: BGR image with blur
    """
    rand_kernel_size = np.random.randint(low=5, high=21)
    rand_kernel = (rand_kernel_size, rand_kernel_size)
    
    return cv2.blur(img_in, rand_kernel)

def aug_shift(img_in):
    """
    Shifts picture in x and y by random (max. 15 % of pixels)
    :param img_in: BGR image (cv2 standard)
    :return: BGR image shifted
    """
    # Percentage
    rand_shift_x = np.random.randint(low=1, high=31) - 15
    rand_shift_y = np.random.randint(low=1, high=31) - 15
    
    # Pixels
    rand_shift_x = np.int(np.shape(img_in)[1] * (rand_shift_x / 100))
    rand_shift_x = np.int(np.shape(img_in)[0] * (rand_shift_y / 100))
    
    # Shift
    shift_m = np.float32([[1,0,rand_shift_x], [0,1,rand_shift_y]])
    img_shift = cv2.warpAffine(img_in, shift_m, (np.shape(img_in)[1], np.shape(img_in)[0]))
    
    return img_shift

def aug_flip(img_in):
    """
    flips image vertically
    :param img_in: BGR image (cv2 standard)
    :return: BRG image flipped
    """
    return np.fliplr(img_in)

def aug_color(img_in):
    """
    Manipulates color channelwise by random (channel and value)
    :param img_in: BGR image (cv2 standard)
    :return: BGR image manipulated in color
    """
    # Convert to float64 array
    img_out = np.array(img_in, dtype=np.float64)
    
    # Random Color Channel and random manipulation
    rand_color = np.random.randint(3)
    rand_manipulation = np.random.randint(low=-20, high=21)
    
    # Manipluation
    img_out[:,:,rand_color] = img_out[:,:,rand_color] + rand_manipulation
    img_out[:,:,rand_color][img_out[:,:,rand_color] > 255] = 255
    img_out[:,:,rand_color][img_out[:,:,rand_color] < 0] = 0
    
    # Convert back and return
    return np.array(img_out, dtype=np.uint8)    

def aug_rotation(img_in):
    """
    Rotates the image
    :param in: BGR image (cv2 standard)
    :return: BRG image rotated
    """
    cols = np.shape(img_in)[1]
    rows = np.shape(img_in)[0]
    
    # Random angle (degrees)
    rand_angle = np.random.randint(low=-10, high=11)
    
    M = cv2.getRotationMatrix2D((cols/2,rows/2), rand_angle, 1)
    dst = cv2.warpAffine(img_in, M, (cols,rows))
    
    return dst

def aug_perspective(img_in):
    """
    Perspective transform of input image
    :param img_in: BGR image (cv2 standard)
    :return: BGR image with perspective transform
    """
    height = np.shape(img_in)[0]
    width = np.shape(img_in)[1]
    
    # Set maximum grab for perspective transform
    percentage_width = np.uint8(0.1 * width)
    percentage_height = np.uint8(0.1 * height)
    
    # Points for transformation
    px1 = [0 + np.random.randint(percentage_width), 0 + np.random.randint(percentage_height)]
    px2 = [width - np.random.randint(percentage_width), 0 + np.random.randint(percentage_height)]
    px3 = [width - np.random.randint(percentage_width), height - np.random.randint(percentage_height)]
    px4 = [0 + np.random.randint(percentage_width), height - np.random.randint(percentage_height)]
    
    org = np.float32([px1, px2, px3, px4])
    
    # Destinations
    dst = np.float32([[0,0], [width, 0], [width, height], [0, height]])
    
    M = cv2.getPerspectiveTransform(org, dst)
    img_out = cv2.warpPerspective(img_in, M, (width, height))
    
    return img_out

In [3]:
[print(np.random.randint(2)) for i in range(10)]

1
0
0
0
1
1
0
1
0
0


[None, None, None, None, None, None, None, None, None, None]

## Examples

In [ ]:
# Brightness Augmentation
for i in range(10):
    plt.imshow(bgr2rgb(aug_brightness(img)))
    plt.axis('off')
    plt.show()

In [ ]:
# Shadow Augmentation
for i in range(10):
    plt.imshow(bgr2rgb(aug_shadow(img)))
    plt.axis('off')
    plt.show()

In [ ]:
# Blur Augmentation
plt.figure()
for i in range(10):
    plt.imshow(bgr2rgb(aug_blur(img)))
    plt.axis('off')
    plt.show()

In [ ]:
# Shift Augmentation
plt.figure()
for i in range(10):
    plt.imshow(bgr2rgb(aug_shift(img)))
    plt.axis('off')
    plt.show()

In [ ]:
# Flip Augmentation
plt.figure()
plt.imshow(bgr2rgb(aug_flip(img)))
plt.axis('off')
plt.show()

In [ ]:
# Color Augmentation
plt.figure()
for i in range(10):
    plt.imshow(bgr2rgb(aug_color(img)))
    plt.axis('off')
    plt.show()

In [ ]:
# Rotation Augmentation
plt.figure()
for i in range(10):
    plt.imshow(bgr2rgb(aug_rotation(img)))
    plt.axis('off')
    plt.show()

In [ ]:
# Perspective Transform Augmentation
plt.figure()
for i in range(10):
    plt.imshow(bgr2rgb(aug_perspective(img)))
    plt.axis('off')
    plt.show()